In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
device = "cuda"
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR, CosineAnnealingWarmRestarts
import torchvision
from sklearn.model_selection import train_test_split
import torchvision.transforms as tt
import torchvision.models as models
from torchvision.datasets import CIFAR10, ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import random_split, DataLoader, Subset, TensorDataset, Dataset
from torch.nn.utils import vector_to_parameters, parameters_to_vector

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

import os
import time
import pickle

from PIL import Image

torch.manual_seed(0)
np.random.seed(0)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip fast-differential-privacy-2.1.zip

Archive:  fast-differential-privacy-2.1.zip
   creating: fast-differential-privacy-2.1/
  inflating: __MACOSX/._fast-differential-privacy-2.1  
  inflating: fast-differential-privacy-2.1/CODE_OF_CONDUCT.md  
  inflating: __MACOSX/fast-differential-privacy-2.1/._CODE_OF_CONDUCT.md  
  inflating: fast-differential-privacy-2.1/LICENSE  
  inflating: __MACOSX/fast-differential-privacy-2.1/._LICENSE  
  inflating: fast-differential-privacy-2.1/requirements.txt  
  inflating: __MACOSX/fast-differential-privacy-2.1/._requirements.txt  
  inflating: fast-differential-privacy-2.1/README.md  
  inflating: __MACOSX/fast-differential-privacy-2.1/._README.md  
   creating: fast-differential-privacy-2.1/fastDP/
  inflating: __MACOSX/fast-differential-privacy-2.1/._fastDP  
  inflating: fast-differential-privacy-2.1/setup.py  
  inflating: __MACOSX/fast-differential-privacy-2.1/._setup.py  
  inflating: fast-differential-privacy-2.1/CONTRIBUTING.md  
  inflating: __MACOSX/fast-differential-privacy-2.

In [4]:
%cd fast-differential-privacy-2.1

/content/fast-differential-privacy-2.1


In [5]:
!python -m setup develop

/content/fast-differential-privacy-2.1
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards

In [6]:
class PathMNIST(Dataset):

    def __init__(self,
                 split='train',
                 transforms=None,
                 target_transform=None,
                 download=False):
        ''' dataset
        :param split: 'train', 'val' or 'test', select subset
        :param transform: data transformation
        :param target_transform: target transformation

        '''

        npz_file = np.load("/content/drive/MyDrive/ISPP_Folder/pathmnist.npz")

        self.split = split
        self.transforms = transforms
        self.transform_index = 0
        self.target_transform = target_transform
        #X = np.concatenate((npz_file['train_images'], npz_file['val_images']))
        #Y = np.concatenate((npz_file['train_labels'], npz_file['val_labels']))
        X_train = npz_file['train_images']
        Y_train = npz_file['train_labels']
        X_val = npz_file['val_images']
        Y_val = npz_file['val_labels']
        X_test = npz_file['test_images']
        Y_test = npz_file['test_labels']
        #print(X.shape)
        #X_train, X_val,Y_train,Y_val = train_test_split(X,Y,test_size = 0.2, random_state =0 )
        print(X_train.shape)
        print(X_val.shape)
        print(X_test.shape)
        if self.split == 'train':  # 89996 images
            self.img = X_train
            self.label = Y_train
        elif self.split == 'val':     #10004  images
            self.img = X_val
            self.label = Y_val
        elif self.split == 'test':      #7180 images
            self.img = X_test
            self.label = Y_test



    def __getitem__(self, index):
        img, target = self.img[index], self.label[index].astype(int)[0]
        img = Image.fromarray(np.uint8(img))

        if self.transforms is not None:
            img = self.transforms[self.transform_index](img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return self.img.shape[0]

In [7]:
light_transform = tt.Compose([
                              tt.ToTensor(),
                              tt.Lambda(lambda x : (x - x.mean())/(x.std()))
])

In [8]:
train_ds = PathMNIST(split='train', transforms= [light_transform])
val_ds = PathMNIST(split='val', transforms= [light_transform])
test_ds = PathMNIST(split='test', transforms= [light_transform])
train_dl = DataLoader(train_ds,512,shuffle = True,num_workers = 4, pin_memory = True)
test_dl = DataLoader(test_ds,512,True,num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_ds,512,True,num_workers = 4, pin_memory = True)

(89996, 28, 28, 3)
(10004, 28, 28, 3)
(7180, 28, 28, 3)
(89996, 28, 28, 3)
(10004, 28, 28, 3)
(7180, 28, 28, 3)
(89996, 28, 28, 3)
(10004, 28, 28, 3)
(7180, 28, 28, 3)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [9]:
!pip install opacus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 5.6 MB/s eta 0:00:00


In [28]:
from fastDP import PrivacyEngine
import torch
import torchvision
torch.manual_seed(2)
import torch.nn as nn
import torch.optim as optim
import timm
from opacus.validators import ModuleValidator
from opacus.accountants.utils import get_noise_multiplier
from tqdm import tqdm
import warnings; warnings.filterwarnings("ignore")

In [29]:
BATCH_SIZE = 512
device = 'cuda'
NUM_CLASSES = 9

In [30]:
import torch
import torch.nn as nn

class CustomBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(CustomBlock, self).__init__()
        # Define conv1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        # Define bn1 (GroupNorm with 2 groups for in_channels channels)
       # self.gn1 = nn.GroupNorm(2, in_channels, eps=1e-05, affine=True)
        # Define ReLU activation
        self.relu = nn.ReLU(inplace=True)
        # Define conv2
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        # Define bn2 (GroupNorm with 2 groups for out_channels channels)
        self.gn2 = nn.GroupNorm(2, out_channels, eps=1e-05, affine=True)
        # Define downsample
        self.downsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=(1, 1), stride=(2, 2), bias=False),
            nn.GroupNorm(32, out_channels, eps=1e-05, affine=True)
        )

    def forward(self, x):
        # Forward pass through conv1, bn1, and relu
        out = self.conv1(x)
       # out = self.gn1(out)
        out = self.relu(out)
        # Forward pass through conv2 and bn2
        out = self.conv2(out)
        out = self.gn2(out)

        # Downsampling path
        residual = self.downsample(x)

        # Add residual to the output
        out += residual

        out = self.conv2(out)
        out = self.gn2(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.gn2(out)


        return out

In [31]:
import torch
import torch.nn as nn

class ResNet_Part1(nn.Module):
    def __init__(self):
        super(ResNet_Part1, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.GroupNorm(32, 64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Layer 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False),
            nn.GroupNorm(2, 64, eps=1e-5, affine=True),
            nn.Conv2d(64, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False),
            nn.GroupNorm(2, 64, eps=1e-5, affine=True),

            nn.Conv2d(64, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False),
            nn.GroupNorm(2, 64, eps=1e-5, affine=True),
            nn.Conv2d(64, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False),
            nn.GroupNorm(2, 64, eps=1e-5, affine=True))


    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)

        return x



In [32]:
import torch
import torch.nn as nn

class ResNet_Part2(nn.Module):
    def __init__(self):
        super(ResNet_Part2, self).__init__()

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 9)

    def forward(self, x):

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x



In [33]:
class FullResNet(nn.Module):
    def __init__(self, model_part1, model_layer2, model_layer3, model_layer4, model_part2):
        super(FullResNet, self).__init__()
        # Store each part as a submodule
        self.model_part1 = model_part1
        self.model_layer2 = model_layer2
        self.model_layer3 = model_layer3
        self.model_layer4 = model_layer4
        self.model_part2 = model_part2

    def forward(self, x):
        # Pass the input through each part sequentially
        x = self.model_part1(x)    # Initial ResNet layers
        x = self.model_layer2(x)   # Custom block layer 2
        x = self.model_layer3(x)   # Custom block layer 3
        x = self.model_layer4(x)   # Custom block layer 4
        x = self.model_part2(x)    # Final ResNet layers
        return x

In [34]:
model_part1 = ResNet_Part1()
model_layer2 = CustomBlock(64, 128)
model_layer3 = CustomBlock(128, 256)
model_layer4 = CustomBlock(256, 512)
model_part2 = ResNet_Part2()

model = FullResNet(model_part1, model_layer2, model_layer3, model_layer4, model_part2)

In [35]:
model = model.to(device)

In [36]:
# for name, module in model.named_modules():
#     print(name, module)

In [37]:
import torch

def get_first_trainable_layer_params(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            return name, param

model = model
name, param = get_first_trainable_layer_params(model)

print("Name:", name)

Name: model_part1.conv1.weight


In [38]:
opt = SGD(model.parameters(),lr = 1e-3)
criterion = nn.CrossEntropyLoss()

In [39]:
BATCH_SIZE = 512
MAX_PHYSICAL_BATCH_SIZE = 512
n_acc_steps = BATCH_SIZE // MAX_PHYSICAL_BATCH_SIZE

In [40]:
print(n_acc_steps)

1


In [41]:
sigma=get_noise_multiplier(
                target_epsilon = 4,
                target_delta = 1e-5,
                sample_rate = BATCH_SIZE/len(train_ds),
                epochs = 1,
            )

In [42]:
print(sigma)

0.60272216796875


In [43]:
privacy_engine = PrivacyEngine(
            model,
            batch_size=BATCH_SIZE,
            max_grad_norm = 40,
            sample_size=len(train_ds),
            target_epsilon = 4,
            target_delta = 1e-5,
            epochs=1,
            clipping_mode='ghost', #"https://pytorch.org/blog/clipping-in-opacus/#:~:text=We%20introduce%20Fast%20Gradient%20Clipping,instantiating%20the%20per%2Dsample%20gradients."
            clipping_style='all-layer',
            origin_params= 'model_part1.conv1.weight',#['patch_embed.proj.bias'],
        )

Using origin parameters for the ghost differentiation trick......
Number of trainable components:  38 ; Number of trainable layers:  26
>>>>>>>>>>>>>>>>> Applying  automatic  per-sample gradient clipping.
>>>>>>>>>>>>>>>>> Block heads for per-sample gradient clipping are defined as: ['model_part1.conv1']


In [44]:
privacy_engine.attach(opt)

In [45]:
def train(epoch):

        model.train()
        train_loss = 0
        correct = 0
        total = 0


        for batch_idx, (inputs, targets) in enumerate(tqdm(train_dl)):
            print(batch_idx)
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            loss.backward()
            if ((batch_idx + 1) % n_acc_steps == 0) or ((batch_idx + 1) == len(train_dl)):
                print("gradient_update",batch_idx)
                opt.step()
                opt.zero_grad()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print('Epoch: ', epoch+1, len(train_dl), 'Train Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        print(privacy_engine.get_privacy_spent())

def test(epoch):
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(tqdm(test_dl)):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

            print('Epoch: ', epoch+1, len(test_dl), 'Test Loss: %.3f | Acc: %.3f%% (%d/%d)'
                             % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

for epoch in range(1):
        train(epoch)

  0%|          | 0/176 [00:00<?, ?it/s]

0


  1%|          | 1/176 [00:01<03:13,  1.11s/it]

gradient_update 0
1


  1%|          | 2/176 [00:01<01:53,  1.53it/s]

...

gradient_update 173
174


 99%|█████████▉| 175/176 [00:49<00:00,  3.80it/s]

gradient_update 174
175
gradient_update 175


100%|██████████| 176/176 [00:49<00:00,  3.57it/s]


Epoch:  1 176 Train Loss: 2.260 | Acc: 12.318% (11086/89996)
{'eps_rdp': 3.9979488851682903, 'alpha_rdp': 3.7}


In [46]:
test(1)

100%|██████████| 15/15 [00:01<00:00,  9.07it/s]

Epoch:  2 15 Test Loss: 2.180 | Acc: 14.749% (1059/7180)


In [47]:
privacy_engine.get_privacy_spent()

{'eps_rdp': 3.9979488851682903, 'alpha_rdp': 3.7}

In [ ]:
torch.save(model.state_dict(),"AWS_Fast_DP_deafult.pt")

In [ ]:
%ls

assets/                 examples/         NOTICE.md         setup.py
AWS_Fast_DP_deafult.pt  fastDP/           __pycache__/      THIRD-PARTY-NOTICES.txt
CODE_OF_CONDUCT.md      fastDP.egg-info/  README.md
CONTRIBUTING.md         LICENSE           requirements.txt
